In [119]:
# required packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap, BoundaryNorm, LinearSegmentedColormap
import warnings
import os
from datetime import datetime

# display option
pd.options.display.float_format = '{:.2f}'.format

## Purpose
Convert the tidy (long) table for export

## Data Load

Load the pickle files from '~/data' directory, along with metrics target list.

In [102]:
df_metric_score_metric_target_miss_streak = pd.read_pickle("../data/case_data_target_miss_streak.pkl")
df_metric_score_metric_underperformance_streak = pd.read_pickle("../data/case_data_performance_miss_streak.pkl")

## Create wide table for target metrics

In [ ]:
def compile_metric_target_info(df, provider):
    
    df = df[df["provider_id"] == provider]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore") 
        df['month_year'] = df['month_year'].dt.strftime('%Y-%m')

    df_pivot_metric = df\
        .pivot(index="metric_source", columns="month_year", values="metric_score_imputed")
    
    df_metric_miss_streak = df.copy()
    df_metric_miss_streak['metric_source'] = 'miss_streak__' + df_metric_miss_streak['metric_source'].astype(str)
    df_metric_miss_streak['metric_score_imputed'] = df_metric_miss_streak['metric_score_imputed'].astype(int)

    df_pivot_met_target = df_metric_miss_streak\
        .pivot(index="metric_source", columns="month_year", values="num_month_streak_metric_missed_target")

    df_concat = pd.concat([df_pivot_metric, df_pivot_met_target])

    df_concat.columns.name = None
    df_concat.reset_index(inplace=True)

    return df_concat


In [104]:
df_metric_score_metric_target_miss_streak_wide = compile_metric_target_info(df_metric_score_metric_target_miss_streak, 1)
df_metric_score_metric_target_miss_streak_wide

,metric_source,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08
0,average_responses_agg_month,0.00,0.00,0.00,0.00,4.70,4.50,4.43,4.64,4.41,4.29,5.00,4.53
1,caseloads_months,235.00,289.00,288.00,282.00,279.00,291.00,315.00,327.00,321.00,319.00,301.00,301.00
2,chart_review_months,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00
3,documentation_rates_months,0.99,1.00,0.99,1.00,0.99,1.00,1.00,0.99,1.00,0.99,1.00,1.00
4,mic_utilization_rate_months,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.06,0.15,0.26,0.22,0.23
5,miss_streak__average_responses_agg_month,1.00,2.00,3.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,miss_streak__caseloads_months,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00,11.00,12.00
7,miss_streak__chart_review_months,1.00,2.00,3.00,4.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
8,miss_streak__documentation_rates_months,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,miss_streak__mic_utilization_rate_months,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00,11.00,12.00


## Create wide table for monthly metrics total performance

In [106]:
def compile_metric_total_performance_info(df, provider):
    
    df = df[df["provider_id"] == provider]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore") 
        df['month_year'] = df['month_year'].dt.strftime('%Y-%m')

    df_pivot_metric = df\
        .pivot(index="provider_id", columns="month_year", values="total_num_metric_target_met")\
        .reset_index(drop=True)
    df_pivot_metric['metric_source'] = "total_num_metric_target_met"


    df_pivot_met_target = df\
        .pivot(index="provider_id", columns="month_year", values="num_month_streak_metric_total_missed_target")\
        .reset_index(drop=True)
    df_pivot_met_target['metric_source'] = "num_month_streak_metric_total_missed_target"

    df_concat = pd.concat([df_pivot_metric, df_pivot_met_target])
    
    cols = ['metric_source'] + [col for col in df_concat.columns if col != 'metric_source']
    df_concat = df_concat[cols]

    df_concat.columns.name = None
    df_concat.reset_index(drop = True, inplace=True)

    return df_concat


In [91]:
df_metric_score_metric_underperformance_streak_wide = compile_metric_total_performance_info(df_metric_score_metric_underperformance_streak, 1)

## Combine target metric and monthly metrics total performance

In [99]:
df_provider_metric_and_performance_summary = pd.concat([df_metric_score_metric_target_miss_streak_wide, df_metric_score_metric_underperformance_streak_wide]).reset_index(drop = True)
df_provider_metric_and_performance_summary

,metric_source,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08
0,average_responses_agg_month,0.00,0.00,0.00,0.00,4.70,4.50,4.43,4.64,4.41,4.29,5.00,4.53
1,caseloads_months,235.00,289.00,288.00,282.00,279.00,291.00,315.00,327.00,321.00,319.00,301.00,301.00
2,chart_review_months,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00
3,documentation_rates_months,0.99,1.00,0.99,1.00,0.99,1.00,1.00,0.99,1.00,0.99,1.00,1.00
4,mic_utilization_rate_months,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.06,0.15,0.26,0.22,0.23
5,miss_streak__average_responses_agg_month,1.00,2.00,3.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,miss_streak__caseloads_months,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00,11.00,12.00
7,miss_streak__chart_review_months,1.00,2.00,3.00,4.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
8,miss_streak__documentation_rates_months,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,miss_streak__mic_utilization_rate_months,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00,11.00,12.00


In [121]:
## Save as csv file

def transform_and_save_provider_report(df_metric, df_performance, provider, export_path, export_filename):
    df_metric_score_metric_target_miss_streak_wide = compile_metric_target_info(df_metric, provider)
    df_metric_score_metric_underperformance_streak_wide = compile_metric_total_performance_info(df_performance, provider)
    df_provider_metric_and_performance_summary = pd.concat([df_metric_score_metric_target_miss_streak_wide, df_metric_score_metric_underperformance_streak_wide]).reset_index(drop = True)

    os.makedirs(export_path, exist_ok=True)

    today = datetime.today()
    year_month_stamp = today.strftime('%Y-%m')
    file_name = f"{export_filename}_{year_month_stamp}_provider_{provider}"
    df_provider_metric_and_performance_summary.to_csv(f'{export_path}/{file_name}.csv')  

    print(f"provider #{provider} report for {year_month_stamp} saved in '{export_path}/{file_name}.csv'")


In [125]:
provider_ids = df_metric_score_metric_target_miss_streak.provider_id.unique()

for provider_id in provider_ids:
    transform_and_save_provider_report(
        df_metric_score_metric_target_miss_streak,
        df_metric_score_metric_underperformance_streak,
        provider_id,
        "../data/provider_export_report",
        "monthly_provider_report"
    )

provider #1 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_1.csv'
provider #2 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_2.csv'
provider #3 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_3.csv'
provider #7 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_7.csv'
provider #8 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_8.csv'
provider #9 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_9.csv'
provider #10 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_10.csv'
provider #11 report for 2025-03 saved in '../data/provider_export_report/monthly_provider_report_2025-03_provider_11.csv'
provider #12 report for 2025-03 save